## Double-DIP Image Dehazing
### Setup

In [ ]:
import os
from dipdehaze import dehaze
from utils.image_io import prepare_image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

input_image_path = "datasets/ohaze/OutdoorTrainHazy/35_outdoor.jpg"
GT_image_path = "datasets/ohaze/OutdoorTrainGT/35_outdoor.jpg"
image_name = os.path.basename(input_image_path)

### Show input image

In [ ]:
input = mpimg.imread(input_image_path)
imgplot = plt.imshow(input)
plt.show()

### Dehazing

In [ ]:
image = prepare_image(input_image_path)
dehaze(image_name, image, num_iter=500, show_every=-1)

### Show dehazed image

In [ ]:
final = mpimg.imread("output/" + image_name + "_final.jpg")
imgplot = plt.imshow(final)
plt.show()

### Evaluate

In [ ]:
import numpy as np
from cv2 import cvtColor, COLOR_RGB2Lab
from skimage import io
from skimage.color import deltaE_ciede2000
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from skimage.transform import resize
from skimage.util import img_as_float32

gt = img_as_float32(io.imread(GT_image_path))
dehazed = img_as_float32(io.imread("output/" + image_name + "_final.jpg"))
# dehazing might have resized the original image, so we need to resize it to match the dehazed image
gt_resized = resize(gt, dehazed.shape, preserve_range=True, anti_aliasing=False)
psnr = peak_signal_noise_ratio(gt_resized, dehazed)
ssim = structural_similarity(gt_resized, dehazed, multichannel=True)
gt_lab = cvtColor(gt_resized, COLOR_RGB2Lab)
in_lab = cvtColor(dehazed, COLOR_RGB2Lab)
deltaE = np.average(deltaE_ciede2000(gt_lab, in_lab))
print("{} - PSNR: {:.6f} SSIM: {:.6f} CIEDE2000: {:.6f}".format(image_name, psnr, ssim, deltaE))